In [2]:
import matplotlib.pyplot as plt 
import numpy as np
import cv2 as cv
import skimage

# Load fits file

In [ ]:
img = skimage.io.fits()

## Galaxy Masking

Something like water shed algorithm 

In [ ]:
masked = mask_galaxy(img)

## Impose min max range 

Lower limit of SNR + ignore pixels saturated by non-galactic source?

In [ ]:
# SNR cut





# Choose saturated pixels





# Mask or what? 

## Try some TMO